In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from  pathlib import Path

@dataclass (frozen= True)
class DataTransformationConfig:
    root_dir: Path 
    data_path : Path

In [ ]:
from mlproj.constants import *
from mlproj.utils.common import read_yaml, create_directories

In [ ]:
class COnfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_file_path= SCHEMA_FILE_PATH):
               
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        data_transformation_config = self.config.data_transformation

        create_directories([data_transformation_config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir= Path(data_transformation_config.root_dir),
            data_path= Path(data_transformation_config.data_path)
        )
        return data_transformation_config

In [ ]:
import os
from mlproj import logger
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_split(self):
        try:
            df = pd.read_csv(self.config.data_path)
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

            train_file_path = self.config.root_dir / "train.csv"
            test_file_path = self.config.root_dir / "test.csv"

            train_df.to_csv(train_file_path, index=False)
            test_df.to_csv(test_file_path, index=False)

            logger.info(f"Train and test data saved at {train_file_path} and {test_file_path} respectively.")

        except Exception as e:
            logger.exception(e)

In [ ]:
try:
    config = COnfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation_config = DataTransformation(config= data_transformation_config)
    data_transformation_config.train_test_split()
except Exception as e:
    logger.exception(e)